In [1]:
from google.colab import files
uploaded = files.upload()

Saving sample_submission.csv.zip to sample_submission.csv.zip
Saving test.csv.zip to test.csv.zip
Saving test_labels.csv.zip to test_labels.csv.zip
Saving train.csv.zip to train.csv.zip


In [70]:
!pip install scikit-multilearn

     |████████████████████████████████| 92kB 2.3MB/s 


In [5]:
from zipfile import ZipFile
# Create a ZipFile Object and load sample.zip in it
with ZipFile('sample_submission.csv.zip', 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall()

In [11]:
!ls

sample_data		   test.csv	    test_labels.csv.zip
sample_submission.csv	   test.csv.zip     train.csv
sample_submission.csv.zip  test_labels.csv  train.csv.zip


In [12]:
#classify the group of sentences as Toxic, severe-toxic, obscene, thread, insult or identity-thread
#Multilabel/Multiclass Classification
#Multilabel: Data can be belong to more than one label 
#Multiclass: Data can not be belong to more than one label
#We are building a multi-label classifier, as a comment can be toxic, obcene and insulting at the same time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline



In [52]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

Using TensorFlow backend.


In [23]:
#Read the csv file into dataframe df
df = pd.read_csv("train.csv")
print(df.shape)

(159571, 8)


In [16]:
#It is observed that running test_train_split on the heavy pre-processed dataset sometimes result in going out of memory. 
# below line causes shuffling of indices, to avoid using train_test_split later
df = df.reindex(np.random.permutation(df.index))

In [25]:
#Seperate the comment-field data and their label 
comment=df["comment_text"]
print(comment.head(10))
#comment = comment.as_matrix()

0    Explanation\nWhy the edits made under my usern...
1    D'aww! He matches this background colour I'm s...
2    Hey man, I'm really not trying to edit war. It...
3    "\nMore\nI can't make any real suggestions on ...
4    You, sir, are my hero. Any chance you remember...
5    "\n\nCongratulations from me as well, use the ...
6         COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK
7    Your vandalism to the Matt Shirvington article...
8    Sorry if the word 'nonsense' was offensive to ...
9    alignment on this subject and which are contra...
Name: comment_text, dtype: object


In [ ]:
label = df[['toxic', 'severe_toxic' , 'obscene' , 'threat' , 'insult' , 'identity_hate']]
print(label.head())

In [59]:
max_features = 20000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(df))
comment_t = tokenizer.texts_to_sequences(comment)
#label_t = tokenizer.texts_to_sequences(label)

In [61]:
maxlen = 250
Comment = pad_sequences(comment_t, maxlen=maxlen)
#Label = pad_sequences(label_t, maxlen=maxlen)

In [63]:
#Splitting data into training and testing
from sklearn.model_selection import train_test_split 
x_train,x_test,y_train,y_test=train_test_split(Comment,label,test_size=0.2)

In [75]:
#Support Vector Machine for Classification Task 
#create and fit classifier
#create and fit classifier
#Binary Relevance Method: Each label is solved seperately like single classification problem
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
classifier = BinaryRelevance(classifier = SVC(), require_dense = [False, True])
classifier.fit(x_train, y_train)

BinaryRelevance(classifier=SVC(C=1.0, break_ties=False, cache_size=200,
                               class_weight=None, coef0=0.0,
                               decision_function_shape='ovr', degree=3,
                               gamma='scale', kernel='rbf', max_iter=-1,
                               probability=False, random_state=None,
                               shrinking=True, tol=0.001, verbose=False),
                require_dense=[False, True])

In [76]:
prediction = classifier.predict(x_test)


In [77]:
#Evaluate the model performance 
from sklearn.metrics import hamming_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss

def evaluate_score(Y_test,predict): 
    loss = hamming_loss(Y_test,predict)
    print("Hamming_loss : ".format(loss*100))
    accuracy = accuracy_score(Y_test,predict)
    print("Accuracy : ".format(accuracy*100))
    try : 
        loss = log_loss(Y_test,predict)
    except :
        loss = log_loss(Y_test,predict.toarray())
    print("Log_loss : ".format(loss))

In [79]:
#evaluate the accuracy of the model 
#calculate scores
evaluate_score(y_test,prediction)

Hamming_loss : 3.7521541594861354
Accuracy : 89.66630111232962
Log_loss : 0.40332132310619917
